In [ ]:
%matplotlib notebook

import ipywidgets as widgets
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl,
    basemaps
)

import datetime as dt
import datacube
import ogr
import matplotlib.pyplot as plt
import rasterio
import xarray
import pandas as pd
import numpy as np

from utils.DEADataHandling import load_clearsentinel2
from utils.utils import transform_from_wgs_poly, three_band_image
from utils.BandIndices import calculate_indices

dc = datacube.Datacube(app='agriculture-app')

def load_agriculture_data():
    today = dt.date.today()
    start_date = today - dt.timedelta(days=120)  # Calculates the time span available for S2a NRT product

    latitude = (-24.974997, -24.995971)
    longitude = (152.429994, 152.395805)

    time = (start_date.strftime("%Y-%m-%d"), today.strftime("%Y-%m-%d"))  # convert dates to strings

    query = {
        'x': longitude,
        'y': latitude,
        'time': time,
        'output_crs': 'EPSG:3577',
        'resolution': (-10,10)
    }

    measurements = (
        'nbar_red',
        'nbar_green',
        'nbar_blue',
        'nbar_nir_1',
        'nbar_swir_2',
        'nbar_swir_3'
    )

    ds_s2 = load_clearsentinel2(
        dc=dc,
        query=query,
        sensors=['s2a', 's2b'],
        product='nrt',
        bands_of_interest=measurements,
        masked_prop=0.0  # keep all images, regardless of pixel quality 
    )
    
    ds_s2['ndvi'], description = calculate_indices(ds_s2, 'NDVI')
    
    return(ds_s2)


def run_agriculture_widget(ds):
    zoom = 14

    # Construct the object that defines the bounding box of the loaded data

    geom_obj = {
        "type": "Feature",
        "properties": {
            "style": {
                "stroke": True,
                "color": 'red',
                "weight": 4,
                "opacity": 0.8,
                "fill": True,
                "fillColor": False,
                "fillOpacity": 0,
                "showArea": True,
                "clickable": True
            }
        },
        "geometry": {
            "type": "Polygon",
            "coordinates": [
                [
                    [
                        152.395805,
                        -24.995971
                    ],
                    [
                        152.395805,
                        -24.974997
                    ],
                    [
                        152.429994,
                        -24.974997
                    ],
                    [
                        152.429994,
                        -24.995971
                    ],
                    [
                        152.395805,
                        -24.995971
                    ]
                ]
            ]
        }
    }

    # create a map geometry from the JSON dictionary (It's a python dict right now. Fix and save as a text file external to the notebook)
    loadeddata_geometry = ogr.CreateGeometryFromJson(str(geom_obj['geometry']))
    center = [loadeddata_geometry.Centroid().GetY(),loadeddata_geometry.Centroid().GetX()]

    # define the study area map
    studyarea_map = Map(center=center, zoom=zoom, basemap=basemaps.Esri.WorldImagery)

    # define the drawing controls
    studyarea_drawctrl = DrawControl(polygon = {"shapeOptions": {"fillOpacity": 0, "color": "blue"}})

    # add drawing controls and data bound geometry to the map
    studyarea_map.add_control(studyarea_drawctrl)
    studyarea_map.add_layer(GeoJSON(data=geom_obj))

    EPSG = 3577

    output_pandas= pd.DataFrame() 
    
    # # function that activates after drawing
    def handle_draw(self, action, geo_json):

        fig = plt.figure(0, figsize=(10,5))
        plt.ylim([-1,1])

        if geo_json['geometry']['type'] == 'Polygon':        
            geom_selectedarea = transform_from_wgs_poly(geo_json['geometry'],EPSG)

            mask = rasterio.features.geometry_mask([geom_selectedarea for geoms in [geom_selectedarea]],
                                                       out_shape=ds.geobox.shape,
                                                       transform=ds.geobox.affine,
                                                       all_touched=False,
                                                       invert=True)

            masked_ds = ds.ndvi.where(mask)

            masked_ds_mean = masked_ds.mean(dim = ['x','y'], skipna=True)
            try:
                masked_ds_mean = masked_ds_mean.interpolate_na(dim = 'time', method = 'nearest')
            except:
                print('Unable to interpolate unknown values.')

            xarray.plot.plot(masked_ds_mean, marker='*')
            
            time = np.ravel(masked_ds_mean.time)
            values = np.ravel(masked_ds_mean.values)
            
            print(time)
            print(values)

    # call to say activate handle_draw function on draw
    studyarea_drawctrl.on_draw(handle_draw)

    # plot the map
    display(studyarea_map)
    print(output_pandas)


In [ ]:
ds_s2 = load_agriculture_data()

In [ ]:
run_agriculture_widget(ds_s2)

In [ ]:
%matplotlib inline

for i in range(14):
    plt.figure(i)
    image = three_band_image(ds_s2, ['nbar_red', 'nbar_green', 'nbar_blue'], time=i)
    plt.imshow(image)